Pasar audio a espectrograma desde el data_set

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Carga del archivo de audio
audio_path = 'ruta/del/audio.wav'  # Cambia esta ruta al archivo que quieres procesar
y, sr = librosa.load(audio_path)

# Cálculo del espectrograma usando la Transformada de Fourier de tiempo corto (STFT)
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

# Visualización del espectrograma
plt.figure(figsize=(12, 8))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Espectrograma')
plt.xlabel("Tiempo")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

def audio_to_spectrogram(audio_path, output_path):
    y, sr = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # Guardar el espectrograma como imagen
    plt.figure(figsize=(2, 2))  # Tamaño para redes neuronales, ajustable
    librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
    plt.axis('off')  # Elimina ejes
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Ejemplo de uso con un archivo
audio_to_spectrogram("ruta/del/audio.wav", "ruta/del/espectrograma.png")


In [ ]:
import pandas as pd

# Supongamos que el CSV tiene columnas 'audio_path' y 'label' (donde label indica COVID o no COVID)
dataset = pd.read_csv('ruta/dataset.csv')
output_dir = "ruta/espectrogramas/"

# Preprocesar cada archivo de audio y guardar los espectrogramas
for idx, row in dataset.iterrows():
    label = row['label']
    audio_path = row['audio_path']
    output_path = os.path.join(output_dir, f"{label}_{idx}.png")
    audio_to_spectrogram(audio_path, output_path)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Generadores de imágenes para entrenamiento y validación
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    output_dir,
    target_size=(64, 64),  # Tamaño de las imágenes
    batch_size=32,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    output_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Definir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_generator, validation_data=validation_generator, epochs=10)
